# CPE018 Midterm Exam (1st Sem, A.Y. 2023-2024)

Student Submission Details:

- Name:
  - Joshua Mico Flores
  - Richmond Ryan Reyes
- Section: CPE31S4
- Schedule: Thurday 1:30PM - 4:30PM
- Instructor: Engr. Verlyn V. Nojor

# Intended Learning Outcomes

By the end of this activity, the student should be able to:

- ILO1: Demonstrate different methods for feature matching and detection learned in class and indepdentently from new sources.
- ILO2: Evaluate the accuracy of different feature matching and detection methods and scrutinize its applicability in solving a given real-life problem.

# Tasks 

For this examination, you must create a mood detection program with an object-oriented programming approach (same as project CAMEO), it must detect mood changes through the use of algorithms/techniques/schemes learned in class, and from external sources.

In this file, you have to include for each section of your solution your completion of the following:

- Part 1: Face Detection: Once your face is detected using any algorithm, it must draw an ROI. The color for the ROI is your choice; however, it must detect for all faces in the frame and draw a corresponding ROI.
- Part 2: Face Recognition: The detected face must then be recognized, using any of the provided tools in class, the ROIs must indicate whether it is your face or someone it doesn't recognize.
- Part 3: Mood Detection: Use three different feature detection and matching techniques to determine three emotion: happy, sad and neutral. Two of the techniques must be learned from class, and 1 must be one you independently learned.

Properly show through your notebook the output for each part of the exam.

# Procedure and Outputs

Notes:

- This is the section where you have to include all your answers to the items provided in the tasks section.
- Tasks 1 and 2 contribute directly to ILO1: Demonstrate different methods for feature matching and detection learned in class and indepdentently from new sources.
- Task 3 contributes directly to ILO2: Evaluate the accuracy of different feature matching and detection methods and scrutinize its applicability in solving a given real-life problem.


# Task 1: Face Detection

In [1]:
# Include your code here
import cv2

def detect():
    face_cascade = cv2.CascadeClassifier(r'C:\Users\reyes\Downloads/haarcascade_frontalface_default.xml')
    eye_cascade = cv2.CascadeClassifier(r'C:\Users\reyes\Downloads/haarcascade_eye.xml')
    camera = cv2.VideoCapture(0)

    while True:
        ret, frame = camera.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            roi_gray = gray[y:y+h, x:x+w]
            colored_frame = frame[y:y+h, x:x+w]
            eyes = eye_cascade.detectMultiScale(roi_gray, 1.2, 5, 0, (40, 40))

        cv2.imshow("camera", frame)
        if cv2.waitKey(int(1000 / 12)) & 0xff == ord("q"):
            break

    camera.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    detect()


# Task 2: Face Recognition

In [ ]:
# Include your code here
import numpy as np
import cv2
import pandas as pd

def read_images(path):
    X, y = [], []
    df = pd.read_csv(path)

    for index, row in df.iterrows():
        filepath, label = str(row['Image Path']), int(row['Label'])
        im = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
        im = cv2.resize(im, (200, 200))
        X.append(np.asarray(im, dtype=np.uint8))
        y.append(label)

    return X, y

def face_rec():
    names = {1: 'richmond', 2: 'mico'}
    X, y = read_images('dataset.csv')

    if not X:
        print("No data found. Please check if the file paths in the CSV file are correct.")
        return

    y = np.asarray(y, dtype=np.int32)

    model = cv2.face.FisherFaceRecognizer_create()
    model.train(X, y)

    camera = cv2.VideoCapture(0)
    face_cascade = cv2.CascadeClassifier(r'C:\Users\Mico\haarcascade_frontalface_default.xml')

    while True:
        ret, img = camera.read()
        if not ret:
            break
        faces = face_cascade.detectMultiScale(img, 1.3, 5)

        for (x, y, w, h) in faces:
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)

            # Extract Region of Interest (ROI)
            roi = img[y:y + h, x:x + w]
            roi = cv2.resize(roi, (200, 200), interpolation=cv2.INTER_LINEAR)
            gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

            params = model.predict(gray_roi)
            print(params)

            cv2.putText(img, f"{names.get(params[0], 'Unknown')}, {params[1]}", (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        cv2.imshow("Face Recognition", img)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    camera.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    face_rec()


# Task 3: Mood Detection

In [4]:
pip install facial-emotion-recognition

Note: you may need to restart the kernel to use updated packages.


In [5]:
# Technique 1

import cv2

import cv2
from facial_emotion_recognition import EmotionRecognition

def detect_mood_from_face(frame, face_cascade, emotion_detector):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]

        # Detect emotion
        emotion, confidence = emotion_detector.predict_emotion(roi_gray)

        # Display the detected emotion on the frame
        cv2.putText(frame, f"Emotion: {emotion} ({confidence:.2f})", (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    return frame

def detect():
    face_cascade = cv2.CascadeClassifier(r'C:\Users\reyes\Downloads/haarcascade_frontalface_default.xml')
    emotion_detector = EmotionRecognition()

    camera = cv2.VideoCapture(0)

    while True:
        ret, frame = camera.read()

        # Perform facial expression detection
        frame = detect_mood_from_face(frame, face_cascade, emotion_detector)

        cv2.imshow("camera", frame)
        if cv2.waitKey(int(1000 / 12)) & 0xff == ord("q"):
            break

    camera.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    detect()

TypeError: EmotionRecognition.__init__() missing 1 required positional argument: 'device'

In [ ]:
# Technique 2

In [ ]:
# Technique 3

# Analysis

For the three different techniques you used in face detection, provide an in-depth analysis.

To do this, you must:

- Test the face detection, face recongition, and mood detection functions 10 times each. Only the mood detection will have components for 10 tests for each different technique used.
- Create a table containing the 10 tests (like shown below) for each task.
- Analyze each output by identifying the accuracy and providing your observations.

In [ ]:
# Import the module for tabulating the data
from tabulate import tabulate
 
# Create a list for content of the table
test_results = [
    ["1", "Happy", "Sad", 0],
    ["2", "Happy", "Neutral", 0],
    ["3", "Happy", "Happy", 1]
]
 
# Create a list for the headers of your table
header = ["Test #", "Expected", "Actual", "Score"]
 
# display table
print("Task 3A: Mood Detection using XYZ Algorithm")
print(tabulate(test_results, headers=header, tablefmt="grid"))

# Calculate for the accuracy
total = 0
for i in test_results:
    total += i[3]
print("Accuracy: ", round(total/len(test_results)*100,2))

# Summary and Lessons Learned

This section must be answered concisely. Do not engage in unmeaningful writing for the summary and lessons learned. Provide your brief reflection only.

**Proprietary Clause**

Property of the Technological Institute of the Philippines (T.I.P.). No part of the materials made and uploaded in this learning management system by T.I.P. may be copied, photographed, printed, reproduced, shared, transmitted, translated or reduced to any electronic medium or machine-readable form, in whole or in part, without prior consent of T.I.P.

Prepared by Engr. RMR